# Import

In [1]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
#import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
directory = 'D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/'
font_path = directory + 'NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name, size=12)
plt.rcParams["figure.figsize"] = (20, 10)
register_matplotlib_converters()

In [2]:
import time
from tqdm import tqdm

# Load Data

In [3]:
dev_users_path ='D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/predict/predict/dev.users'
dev_users_list = []
with open(dev_users_path, 'r') as fr:
    lines = fr.readlines()
    dev_users_list = lines
    del lines
for i in range(len(dev_users_list)):
    dev_users_list[i] = dev_users_list[i].replace('\n','')
    
test_users_path ='D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/predict/predict/test.users'
test_users_list = []
with open(test_users_path, 'r') as fr:
    lines = fr.readlines()
    test_users_list = lines
    del lines
for i in range(len(test_users_list)):
    test_users_list[i] = test_users_list[i].replace('\n','')

print(len(dev_users_list))
print(len(test_users_list))

3000
5000


In [ ]:
- FEB_top_30000 : 2월 1일 ~ 3월 1일 읽은 글 top 30000
- TOTAL_top_30000 : 10월 1일 ~ 3월 1일까지 읽은 글 top 30000

- FEB_top_58647 : 2월 14일 ~ 3월 1일간 읽은 글 best
    
- FEB_MAR_article : 2월 1일 ~ 3월 까지 등록된 글
- FEB_MAR_article_2 : 2월 14일 ~ 3월 1일 등록된 글
- MAR_article_1 : 3월 1일 ~ 3월 7일 등록된 글
- MAR_article_2 : 3월 7일 ~ 3월 14일 등록된 글 
 *기간은 왼쪽 처음은 닫힌 구간, 오른쪽 끝은 열린 구간

In [4]:
FEB_top_58647 = pd.read_csv(directory+'FEB_top_58647.csv')
FEB_top_58647 = FEB_top_58647.drop('Unnamed: 0',1)
FEB_MAR_article_2 = pd.read_csv(directory+'FEB_MAR_article_2.csv')
FEB_MAR_article_2 = FEB_MAR_article_2.drop('Unnamed: 0',1)
MAR_article_1 = pd.read_csv(directory+'MAR_article_1.csv')
MAR_article_1 = MAR_article_1.drop('Unnamed: 0',1)
MAR_article_2 = pd.read_csv(directory+'MAR_article_2.csv')
MAR_article_2 = MAR_article_2.drop('Unnamed: 0',1)

print(len(FEB_top_30000))
print(len(FEB_MAR_article))
print(len(TOTAL_top_30000))

print('20190723 update')
print(len(FEB_top_58647))
print(len(FEB_MAR_article_2)) # 이건 필요가 없을 수도 있겠다 어차피 안 팔릴 글은 안 읽었을테니
print(len(MAR_article_1))
print(len(MAR_article_2))


30000
44278
30000
20190723 update
58648
11641
4654
5322


# 읽은 글 체크

In [ ]:
- article_seen_by_testuser

In [5]:
with open(directory + 'article_by_testuser.json') as f:
    article_seen_by_testuser = json.load(f)

## following list

In [7]:
users = pd.read_json(directory + '/users.json', lines=True)

In [8]:
users[:1]

,following_list,id,keyword_list
0,"[@perytail, @brunch]",#901985d8bc4c481805c4a4f911814c4a,[]


In [9]:
import time
st = time.time()
users_following_lists = [] 
for i in range(5000):
    uid = test_users_list[i]
    ufl = list(users[users['id'] == uid]['following_list'])
    if len(ufl) > 0:
        users_following_lists.append(ufl)
    else:
        users_following_lists.append([])
print(time.time() - st , 'sec')

In [10]:
len(users_following_lists)

5000

In [12]:
users_following_lists[0][0]

['@avecrhae',
 '@gongdae-unni',
 '@jhcharm1',
 '@travel-heather',
 '@swimmingstar',
 '@1simforyou',
 '@alicemelbourne',
 '@2jaemyung',
 '@brunch']

- r2
- author_lists
- class_lists
- magainze_lists
- unique_author

In [ ]:
# 20190207 이후에 읽은 글들로 앞으로 읽을 글들을 추천

In [13]:
with open(directory + 'rasby_devuser_100.csv',encoding='utf-8') as f:
    r1 = pd.read_csv(f)

r1 = r1.drop('Unnamed: 0',1)
r1['read_dt']  = r1['read_dt'].map(lambda x : x.replace('-',''))
a = '20190207' # 이것도 hyp 
r2 = r1[r1['read_dt'] >= a]

def int_class(x):
    if x == '5%':
        result = 5
    elif x == '10%':
        result = 10
    elif x == '25%':
        result = 25
    elif x == '50%':
        result = 50
    elif x == '75%':
        result = 75
    else:
        result = 100
    return result

In [14]:
r2[:2]

,read_dt,user_id,article_id,title,sub_title,author_id,reg_dt,type,display_url,keyword_list,magazine_id,off_day,read_cnt,class
0,20190212,#d6866a498157771069fdf15361cb012b,@april_78,"주택청약, 저는 언제쯤 제 집을...?",NaN,@april,2019-02-06,매거진,https://brunch.co.kr/@april/78,"['주택청약통장', '주택청약', '청약통장']",42262.0,6.0,300.0,5%
1,20190212,#d6866a498157771069fdf15361cb012b,@noonsil_12,틀린 게 아니고 다른 겁니다.,주부의 눈으로 본 태국이 다른 점,@noonsil,2019-02-08,매거진,https://brunch.co.kr/@noonsil/12,"['에어비앤비', '치앙마이', '파타야']",42485.0,4.0,703.0,5%


In [ ]:
# 평균 중복소비가 20건
# 평가 데이터에는 중복 소비가 없음
# 따라서 중복 소비 제거하고 최신 동향으로 추천하는 식 ㄱㄱ

In [15]:
# 읽은 구독자 list 도 만들기
# 개인 or magazine
# 이부분은 나중에 test_users_list , 5000 , test_user에 맞게 수정해주어야함
author_lists = []
class_lists = []
magazine_lists = []
unique_author = []
for i in tqdm(range(3000),mininterval = 1): ### final 시 수정
    recent_read_raw = r2[r2['user_id'] == dev_users_list[i]]  
    recent_read_raw = recent_read_raw.sort_values(by = ['read_dt'],ascending = False)
    recent_read_raw = recent_read_raw.drop_duplicates(['article_id'])
    
    len_recent_read_raw = len(recent_read_raw)
    
    # class mean
    class_sum = 0
    magazine_sum = 0
    # 최신에 읽은 작가들 정보 모으기
    author_list = []
    author_cnt = []
    for j in range(len_recent_read_raw):
        # class mean
        rrr = recent_read_raw[j:j+1]
        class_int = int_class(list(rrr['class'])[0])
        class_sum += class_int
        # magainze mean
        magazine_type = list(rrr['type'])[0]
        if(magazine_type == '매거진'):
            magazine_sum += 1
        # author list
        author = list(rrr['author_id'])[0]
        found = 0
        for k in range(len(author_list)):
            if author_list[k] == author:
                author_cnt[k] += 1
                found = 1
                break
        if(found == 0):
            author_list.append(author)
            author_cnt.append(1)

    author_list_dict = dict()
    for j in range(len(author_list)):
        author_list_dict[author_list[j]] = author_cnt[j]

    if(len_recent_read_raw != 0 ):
        class_mean = class_sum / len_recent_read_raw
        class_lists.append(class_mean)
        
        magazine_mean = magazine_sum / len_recent_read_raw
        magazine_lists.append(magazine_mean)
        
        #author_list_dict = sorted(author_list_dict.items(), key=(lambda x: x[1]), reverse = True) # 이러면 [ (), () ]형태
        author_lists.append(author_list_dict)
        
        unique_author.append(len(author_list))
    else:
        class_lists.append(5.0)
        magazine_lists.append(1.0)
        author_lists.append(dict())
        unique_author.append(0)



100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [01:06<00:00, 45.10it/s]


In [ ]:
# 아래 얘네들 키 안 맞음

In [100]:
# 최신에 읽은 글들에서 구독자들이 있는지 확인
# 없는 구독자는 이때까지 읽은 글을 검사해서 최신 동향이 아닌 구독자는 빼면 더 좋을 듯 하긴하네
diff_fa = []
diff_cnt = 0
for i in range(3000):
    keys = set(list(author_lists[i].keys()))
    for j in range(5000):
        if uid == test_users_list[j]:
            i = j
            break
    if len(users_following_lists[i]) > 0:
        following_author = set(users_following_lists[i][0])
    else :
        following_author = set()
    diff_fa = list(following_author - keys)
    diff_cnt += len(diff_fa)


In [101]:
diff_cnt / 3000

1.612

In [20]:
following_author_cnt = 0
for i in range(3000):
    if len(users_following_lists[i]) > 0:
        following_author_cnt += len(users_following_lists[i][0])

In [21]:
following_author_cnt /3000

27.88233333333333

In [ ]:
author_lists 
class_lists 
magazine_lists 
unique_author 

# SIM 기존 버전

In [73]:
article_similarity = np.load(directory+'article_similarity_t.npy')
article_similarity = article_similarity.tolist()
with open(directory + 'word_to_id.json') as f:
    word_to_id  = json.load(f)

with open(directory + 'id_to_word.json') as f:
    id_to_word = json.load(f)

print(len(article_similarity))
print(len(word_to_id))
print(len(id_to_word))

134994
505842
134994


# SIM 업데이트 버전

In [47]:
article_similarity = np.load(directory+'article_similarity_recent_t.npy')
article_similarity = article_similarity.tolist()
with open(directory + 'word_to_id_recent.json') as f:
    word_to_id  = json.load(f)

with open(directory + 'id_to_word_recent.json') as f:
    id_to_word = json.load(f)

print(len(article_similarity))
print(len(word_to_id))
print(len(id_to_word))

165110
486427
165110


# Dev User

In [48]:
FEB_top_58647_list = FEB_top_58647['article_id'].tolist()

In [54]:
r[:3]

['@seungyae613_22', '@namgizaa_46', '@connerstoneiqvk_26']

In [55]:
r = article_seen_by_testuser[str(0)]

In [57]:
r = article_seen_by_testuser[str(0)]
r = list(reversed(r))
s = []
for a in range(len(r)):
    if r[a] not in s:
        s.append(r[a])
r = list(reversed(s))

In [53]:
len(r)
type(r)

list

In [ ]:
r = article_seen_by_testuser[str(i)]
        
# 읽은 아티클에서 중복 제거할떄도 최신의 글이 살아남게끔 제거해야한다
s = []
for a in range(len(r)):
    if r[a] not in s:
        s.append(r[a])
r = s


# 좀더 신중히 다뤄 보도록 하자

In [78]:
FEB_top_58647[30410:30420]

,article_id,read_cnt,id,display_url,keyword_list,magazine_id,reg_ts,sub_title,title,author_id,reg_datetime,reg_dt,type,class
30410,@dooook_141,10,141.0,https://brunch.co.kr/@dooook/141,"['메신저', '왓츠앱', '카카오톡']",19396.0,1.547551e+12,글로벌 메신저 '왓츠앱'과 한국의 국민 메신저 '카카오톡',글로벌한 시각은 글로벌한 경험에서 나온다,@dooook,2019-01-15 20:09:50,2019-01-15,매거진,50%
30411,@lunar611_7,10,7.0,https://brunch.co.kr/@lunar611/7,"['작가지망생', '아르바이트', '친구']",41194.0,1.547053e+12,당신을 묵묵히 응원해줄 단 한 사람,6. 지망생의 여행가방에 꼭 넣어야 할 것들 (2),@lunar611,2019-01-10 01:49:17,2019-01-10,매거진,50%
30412,@zzyoun_46,10,46.0,https://brunch.co.kr/@zzyoun/46,"['영화', '택시운전사', '송강호']",20709.0,1.507544e+12,<택시운전사>,직업윤리와 나비효과,@zzyoun,2017-10-09 19:21:23,2017-10-09,매거진,50%
30413,@honeytip_1109,10,1109.0,https://brunch.co.kr/@honeytip/1109,"['맛집', '강릉', '여행']",0.0,1.540627e+12,NaN,강릉 여행에서 꼭 가봐야 할 유명 맛집 6곳,@honeytip,2018-10-27 16:48:55,2018-10-27,개인,50%
30414,@hosungsong_380,10,380.0,https://brunch.co.kr/@hosungsong/380,"['사진', '웨딩스튜디오추천']",37837.0,1.546064e+12,사람,#18,@hosungsong,2018-12-29 15:18:06,2018-12-29,매거진,50%
30415,@jjacksarang_61,10,61.0,https://brunch.co.kr/@jjacksarang/61,"['이별', '사랑', '연애']",25672.0,1.513382e+12,내가 아는 '그'의 이야기,착함 콤플렉스,@jjacksarang,2017-12-16 08:48:17,2017-12-16,매거진,50%
30416,@jacobsfoto_125,10,125.0,https://brunch.co.kr/@jacobsfoto/125,"['사진', '사랑', '이별']",3493.0,1.546701e+12,NaN,가치있어서가 아니라 같이있어서,@jacobsfoto,2019-01-06 00:04:04,2019-01-06,매거진,50%
30417,@hotaro_10,10,10.0,https://brunch.co.kr/@hotaro/10,"['온라인쇼핑몰', '쇼핑몰', '시크헤라']",38301.0,1.540639e+12,3세대 쇼핑몰의 빅데이터 전략 엿보기,온라인쇼핑몰 '시크헤라' 인터뷰,@hotaro,2018-10-27 20:19:26,2018-10-27,매거진,50%
30418,@movemovemove_24,10,24.0,https://brunch.co.kr/@movemovemove/24,"['세계여행', '여행', '남미']",12170.0,1.495759e+12,남미 그리고 다시 북미로,"세계여행 9개월, 3/4분기 정산",@movemovemove,2017-05-26 09:38:17,2017-05-26,매거진,50%
30419,@ccantare_2,9,2.0,https://brunch.co.kr/@ccantare/2,"['그림일기', '애니메이션', '가족']",43107.0,1.550302e+12,시작합니다!,#1 시작합니다~,@ccantare,2019-02-16 16:24:34,2019-02-16,매거진,50%


In [ ]:
# FEB_top_30419 , 비교 필요

In [81]:
# read_cnt >= 10 만 자른것
FEB_top_30419 = FEB_top_58647[:30419]
FEB_top_30419_list = FEB_top_30419['article_id'].tolist()
len(FEB_top_30419)

In [ ]:
# 일단 가장 베스트

In [94]:
#hyper parameter
hyp_author = 100
hyp_recent_article_len = 10
hyp_top_k = 10
hyp_read_cnt = 10
hyp_sim = 80

isdev = 1

hyp = str(hyp_author) + '_' +str(hyp_recent_article_len)+'_'+str(hyp_top_k)+'_'+ str(hyp_read_cnt)+'_'+str(hyp_sim)+'_'+'DEV_FINAL'

try:
    if not(os.path.isdir(directory + hyp )):
        os.makedirs(os.path.join(directory + hyp))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directory!!!!!")
        raise
            
countt = 0
cntttt = 0

ral = []
siml = []
tol = []
ulfs = []
tol2 = []
with open(directory + hyp +'/recommend' + '.txt','w') as file:
    for i in tqdm(range(len(dev_users_list)),mininterval=10):
        
        recommend_all=[]
        uid = dev_users_list[i]
        aid = i # author index 
        
        for j in range(5000):
            if uid == test_users_list[j]:
                i = j
                break
        
        # 현재 유저가 읽은 following list
        ufl = users_following_lists[i]
        if(len(ufl) > 0):
            ufl  = ufl[0]
            
        # 현재 유저가 읽은 모든 글 , 
        r = article_seen_by_testuser[str(i)]
        # 최신에 읽은 글들이 사라지지 않게 뒤집고 중복 제거하기
        r = list(reversed(r))
        s = []
        
        for a in range(len(r)):
            if r[a] not in s:
                s.append(r[a])
        r = list(reversed(s))

        # 최신 읽은 작가 정보를 업데이트
        # type(author_and_cnt) = dict 임
        author_and_cnt = author_lists[aid]
        if hyp_author > 0:
            
            # 일단 recent 2주 top 58647 에서
            recommend_author = []
            cnt_author = 0            
            author_consider_set = set()
            
            for key, value in author_and_cnt.items(): #이거를 3개 만들어서 하는 게 NDCG 높이는데 기여할듯
                author = key
                cnt = value
                author_consider_set.add(key)
                # 2월 14일 ~ 3월 1일간 인기글 30419개 중에 있으면 뽑기
                author_all_article_1 = FEB_top_30419[FEB_top_30419['author_id'] == author].sort_values(by='reg_dt',ascending=False) # 최신에 작성한 글들의 인기순이겠지 # 더 줄여봐도 될 듯
                cnt_1 = 0
                for j in range(len(author_all_article_1)):
                    if(cnt_1 >= cnt): break
                    artic = list(author_all_article_1[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_author:
                            recommend_author.append(artic)
                            cnt_1 +=1
                
                #  3월 1일 ~ 3월 7일에 쓰여진 글 있으면 append
                author_all_article_2 = MAR_article_1[MAR_article_1['author_id'] == author]
                cnt_2 = 0
                for j in range(len(author_all_article_2)):
                    if(cnt_2 >= cnt): break
                    artic = list(author_all_article_2[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_author:
                            recommend_author.append(artic)
                            cnt_2 +=1
                            
                #  3월 7일 ~ 3월 14일에 쓰여진 글 있으면 append
                author_all_article_3 = MAR_article_2[MAR_article_2['author_id'] == author]
                cnt_3 = 0
                for j in range(len(author_all_article_3)):
                    if(cnt_3 >= cnt): break
                    artic = list(author_all_article_3[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_author:
                            recommend_author.append(artic)
                            cnt_3 +=1
                
            
            # 100 개 이상 추천될 수 있으니 자르기 | hyper parameter 만큼 자르기
            recommend_author = recommend_author[:hyp_author] # 체크
            ral.append(len(recommend_author))
            for artic in recommend_author:
                if artic not in recommend_all:
                    recommend_all.append(artic)
            
        # 구독자를 먼저 넣어보자
        # recent때 활용하지 않은 구독자 정보를 이용한다
        ufl_not_yet = []
        for t in range(len(ufl)):
            if ufl[t] not in author_consider_set:
                ufl_not_yet.append(ufl[t])

        ufls_i = 0
        if len(ufl_not_yet) > 0:

            for author in ufl_not_yet: # 여기도 제일 많이 읽은 작가 순으로 넣어주는게 바람직하겠지?
                if(len(recommend_all) >= 100): break
                author_all_article_1 = FEB_top_30419[FEB_top_30419['author_id'] == author].sort_values(by='reg_dt',ascending=False) # top 50000 or to 10000 으로 바꿔서 비

                cnt_1 = 0
                for j in range(len(author_all_article_1)):
                    if(len(recommend_all) >= 80): break #이것도 조정 필요

                    artic = list(author_all_article_1[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_all:
                            recommend_all.append(artic)
                            cnt_1 +=1
                            ufls_i+=1


                #  3월 1일 ~ 3월 7일에 쓰여진 글 있으면 append

                author_all_article_2 = MAR_article_1[MAR_article_1['author_id'] == author]
                cnt_2 = 0
                for j in range(len(author_all_article_2)):
                    if(len(recommend_all) >= 90): break #얘도 조정 필요하고
                    artic = list(author_all_article_2[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_all:
                            recommend_all.append(artic)
                            cnt_2 +=1
                            ufls_i+=1

                #  3월 7일 ~ 3월 14일에 쓰여진 글 있으면 append
                author_all_article_3 = MAR_article_2[MAR_article_2['author_id'] == author]
                cnt_3 = 0
                for j in range(len(author_all_article_3)):
                    if(len(recommend_all) >= 100): break #얘는 그대로 뭐 둬도 (entropy가 높지 않으면 바로 아래 걸로)
                    artic = list(author_all_article_3[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_all:
                            recommend_all.append(artic)
                            cnt_3 +=1
                            ufls_i+=1
        tol.append(len(recommend_all))
        
        # 최신에 본 글 위주로 similarity 조사
        if(len(r) >= hyp_read_cnt):
            wtil = list(word_to_id.keys())
            countt +=1
            if(hyp_recent_article_len > len(r)):
                hyp_recent_article_len = len(r)
            r = r[-hyp_recent_article_len:]
            r = list(reversed(r))
            siml_cnt = 0
            for j in range(len(r)):
                if(siml_cnt>= hyp_sim): break
                if(r[j] not in wtil):
                    r[j] = 'UNK'
                s = article_similarity[word_to_id[r[j]]]
                sim_list = s[:hyp_top_k]
                for k in range(len(sim_list)):
                    if sim_list[k] not in recommend_all:
                        recommend_all.append(sim_list[k])
                        siml_cnt+=1
                        if(siml_cnt>= hyp_sim): break
            siml.append(siml_cnt)
            if len(recommend_all)>=100:
                recommend_all = recommend_all[:100]
        
        
        tol2.append(len(recommend_all))
        
        if(len(recommend_all) >= 100):
            recommend_all = recommend_all[:100]
        else:
            # 남으면 FEB_top 중 아무거나 뽑아서 주고
            # following list 있으면 여기에 활용해도 좋을듯
            cntttt+=1
            
                    
            for x in FEB_top_30419_list:
                if x not in recommend_all:
                    recommend_all.append(x)
                    if(len(recommend_all) == 100): break
                        
        # 다시 한 번 더 점검
        if(len(recommend_all) > 100):
            recommend_all = recommend_all[:100]
        

        recommend_normal = ''
        for x in recommend_all:
            recommend_normal += ' '+ x

        file.write(uid + recommend_normal)
        file.write('\n')



100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [08:20<00:00,  5.99it/s]

In [95]:
print(sum(ral)/3000)
print(sum(siml)/len(siml))
print(sum(tol)/3000)
print(sum(tol2)/3000)
print(cntttt)

46.34133333333333
76.28275067230119
82.17266666666667
95.77
478


In [ ]:
# 0.071237	0.166344	9.024134

- ral = author 로 추천된 아티클 수
- siml = similarity 로 추천된 아티클 수
- tol = 두개 합쳐서 전체 추천된 아티클 수
- cntttt = else 에 걸리는 부분 

In [ ]:
# 또 해볼 것들


In [ ]:
#hyper parameter
hyp_author = 100
hyp_recent_article_len = 10
hyp_top_k = 10
hyp_read_cnt = 10
hyp_sim = 80

isdev = 1

hyp = str(hyp_author) + '_' +str(hyp_recent_article_len)+'_'+str(hyp_top_k)+'_'+ str(hyp_read_cnt)+'_'+str(hyp_sim)+'_'+'DEV_FINAL_58647'

try:
    if not(os.path.isdir(directory + hyp )):
        os.makedirs(os.path.join(directory + hyp))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directory!!!!!")
        raise
            
countt = 0
cntttt = 0

ral = []
siml = []
tol = []
ulfs = []
tol2 = []
with open(directory + hyp +'/recommend' + '.txt','w') as file:
    for i in tqdm(range(len(dev_users_list)),mininterval=10):
        
        recommend_all=[]
        uid = dev_users_list[i]
        aid = i # author index 
        
        for j in range(5000):
            if uid == test_users_list[j]:
                i = j
                break
        
        # 현재 유저가 읽은 following list
        ufl = users_following_lists[i]
        if(len(ufl) > 0):
            ufl  = ufl[0]
            
        # 현재 유저가 읽은 모든 글 , 
        r = article_seen_by_testuser[str(i)]
        # 최신에 읽은 글들이 사라지지 않게 뒤집고 중복 제거하기
        r = list(reversed(r))
        s = []
        
        for a in range(len(r)):
            if r[a] not in s:
                s.append(r[a])
        r = list(reversed(s))

        # 최신 읽은 작가 정보를 업데이트
        # type(author_and_cnt) = dict 임
        author_and_cnt = author_lists[aid]
        if hyp_author > 0:
            
            # 일단 recent 2주 top 58647 에서
            recommend_author = []
            cnt_author = 0            
            author_consider_set = set()
            
            for key, value in author_and_cnt.items(): #이거를 3개 만들어서 하는 게 NDCG 높이는데 기여할듯
                author = key
                cnt = value
                author_consider_set.add(key)
                # 2월 14일 ~ 3월 1일간 인기글 30419개 중에 있으면 뽑기
                author_all_article_1 = FEB_top_58647[FEB_top_58647['author_id'] == author].sort_values(by='reg_dt',ascending=False) # 최신에 작성한 글들의 인기순이겠지 # 더 줄여봐도 될 듯
                cnt_1 = 0
                for j in range(len(author_all_article_1)):
                    if(cnt_1 >= cnt): break
                    artic = list(author_all_article_1[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_author:
                            recommend_author.append(artic)
                            cnt_1 +=1
                
                #  3월 1일 ~ 3월 7일에 쓰여진 글 있으면 append
                author_all_article_2 = MAR_article_1[MAR_article_1['author_id'] == author]
                cnt_2 = 0
                for j in range(len(author_all_article_2)):
                    if(cnt_2 >= cnt): break
                    artic = list(author_all_article_2[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_author:
                            recommend_author.append(artic)
                            cnt_2 +=1
                            
                #  3월 7일 ~ 3월 14일에 쓰여진 글 있으면 append
                author_all_article_3 = MAR_article_2[MAR_article_2['author_id'] == author]
                cnt_3 = 0
                for j in range(len(author_all_article_3)):
                    if(cnt_3 >= cnt): break
                    artic = list(author_all_article_3[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_author:
                            recommend_author.append(artic)
                            cnt_3 +=1
                
            
            # 100 개 이상 추천될 수 있으니 자르기 | hyper parameter 만큼 자르기
            recommend_author = recommend_author[:hyp_author] # 체크
            ral.append(len(recommend_author))
            for artic in recommend_author:
                if artic not in recommend_all:
                    recommend_all.append(artic)
            
        # 구독자를 먼저 넣어보자
        # recent때 활용하지 않은 구독자 정보를 이용한다
        ufl_not_yet = []
        for t in range(len(ufl)):
            if ufl[t] not in author_consider_set:
                ufl_not_yet.append(ufl[t])

        ufls_i = 0
        if len(ufl_not_yet) > 0:

            for author in ufl_not_yet: # 여기도 제일 많이 읽은 작가 순으로 넣어주는게 바람직하겠지?
                if(len(recommend_all) >= 100): break
                author_all_article_1 = FEB_top_58647[FEB_top_58647['author_id'] == author].sort_values(by='reg_dt',ascending=False) # top 50000 or to 10000 으로 바꿔서 비

                cnt_1 = 0
                for j in range(len(author_all_article_1)):
                    if(len(recommend_all) >= 80): break #이것도 조정 필요

                    artic = list(author_all_article_1[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_all:
                            recommend_all.append(artic)
                            cnt_1 +=1
                            ufls_i+=1


                #  3월 1일 ~ 3월 7일에 쓰여진 글 있으면 append

                author_all_article_2 = MAR_article_1[MAR_article_1['author_id'] == author]
                cnt_2 = 0
                for j in range(len(author_all_article_2)):
                    if(len(recommend_all) >= 90): break #얘도 조정 필요하고
                    artic = list(author_all_article_2[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_all:
                            recommend_all.append(artic)
                            cnt_2 +=1
                            ufls_i+=1

                #  3월 7일 ~ 3월 14일에 쓰여진 글 있으면 append
                author_all_article_3 = MAR_article_2[MAR_article_2['author_id'] == author]
                cnt_3 = 0
                for j in range(len(author_all_article_3)):
                    if(len(recommend_all) >= 100): break #얘는 그대로 뭐 둬도 (entropy가 높지 않으면 바로 아래 걸로)
                    artic = list(author_all_article_3[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_all:
                            recommend_all.append(artic)
                            cnt_3 +=1
                            ufls_i+=1
        tol.append(len(recommend_all))
        
        # 최신에 본 글 위주로 similarity 조사
        if(len(r) >= hyp_read_cnt):
            wtil = list(word_to_id.keys())
            countt +=1
            if(hyp_recent_article_len > len(r)):
                hyp_recent_article_len = len(r)
            r = r[-hyp_recent_article_len:]
            r = list(reversed(r))
            siml_cnt = 0
            for j in range(len(r)):
                if(siml_cnt>= hyp_sim): break
                if(r[j] not in wtil):
                    r[j] = 'UNK'
                s = article_similarity[word_to_id[r[j]]]
                sim_list = s[:hyp_top_k]
                for k in range(len(sim_list)):
                    if sim_list[k] not in recommend_all:
                        recommend_all.append(sim_list[k])
                        siml_cnt+=1
                        if(siml_cnt>= hyp_sim): break
            siml.append(siml_cnt)
            if len(recommend_all)>=100:
                recommend_all = recommend_all[:100]
        
        
        tol2.append(len(recommend_all))
        
        if(len(recommend_all) >= 100):
            recommend_all = recommend_all[:100]
        else:
            # 남으면 FEB_top 중 아무거나 뽑아서 주고
            # following list 있으면 여기에 활용해도 좋을듯
            cntttt+=1
            
                    
            for x in FEB_top_58647_list:
                if x not in recommend_all:
                    recommend_all.append(x)
                    if(len(recommend_all) == 100): break
                        
        # 다시 한 번 더 점검
        if(len(recommend_all) > 100):
            recommend_all = recommend_all[:100]
        

        recommend_normal = ''
        for x in recommend_all:
            recommend_normal += ' '+ x

        file.write(uid + recommend_normal)
        file.write('\n')



  5%|███▊                                                                           | 144/3000 [00:32<10:25,  4.57it/s]